In [70]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.search.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/code-samples#python

#map used to plot circles:
# https://www.mapdevelopers.com/draw-circle-tool.php?circles=%5B%5B1000000%2C31.8986083%2C-103.3465565%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B1000000%2C41.5647182%2C-116.3364002%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B1000000%2C33.4940108%2C-84.3442127%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B1000000%2C46.0955066%2C-94.5818799%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%2C%5B1000000%2C44.9253219%2C-71.7303174%2C%22%23AAAAAA%22%2C%22%23000000%22%2C0.4%5D%5D

import os
import googleapiclient.discovery
import pandas as pd
import json

BASE_DIR = '../scraping'
KEY_PATH = os.path.join(BASE_DIR, "api_keyPG.txt")

with open(KEY_PATH) as f:
    api_key = f.readline()



In [65]:


def request_search_results(token='',region_center='31.898608,-103.346556'):
    '''requests 50 search results for a specified query using the Youtube Data API V3, returns a complex dictionary'''
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = api_key

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.search().list(
        part="snippet",
        maxResults=50,
        topicId="/m/032tl | /m/01k8wb | /m/027x7n | /m/02wbm",
        pageToken=token,
        q="how to | tutorial | recipe | step | lesson | guide | demo | DIY",
        type="video",
        videoCategoryId="26",
        regionCode = 'US',
        location = region_center,
        locationRadius = "1000km"
    )
    response = request.execute()

    return response

In [ ]:
def get_vid_ids(dict_list):
    
    id_list = []

    for i in range(len(dict_list)):
        vid_info = {
        'videoID' : dict_list[i]['id']['videoId'],
        'title' : dict_list[i]['snippet']['title']
}
        id_list.append(vid_info)
    return id_list


In [64]:
def get_tutorial_url_list(loop_len=1,track=True):
    '''Calls functions to request youtube search results as a list of video IDs and titles as a dictionary, with 50 IDs per iteration'''
    page_token = ''
    full_id_list = []

    #define region centers:
    west = '41.564718,-116.336400'
    west_page = ''
    texas = '31.898608,-103.346556'
    texas_page = ''
    midwest = '46.095507,-94.581880'
    midwest_page = ''
    southeast = '33.494011,-84.344213'
    southeast_page = ''
    northeast = '44.925322,-71.730317'
    northeast_page = ''
    
    print("Number of results so far:")
    
    for i in range(loop_len):
        west_soup = request_search_results(token=west_page,region_center=west)
        west_page = west_soup['nextPageToken']
        west_list = west_soup['items']

        texas_soup = request_search_results(token=texas_page,region_center=texas)
        texas_page = texas_soup['nextPageToken']
        texas_list = texas_soup['items']

        midwest_soup = request_search_results(token=midwest_page,region_center=midwest)
        midwest_page = midwest_soup['nextPageToken']
        midwest_list = midwest_soup['items']

        southeast_soup = request_search_results(token=southeast_page,region_center=southeast)
        southeast_page = southeast_soup['nextPageToken']
        southeast_list = southeast_soup['items']

        northeast_soup = request_search_results(token=northeast_page,region_center=northeast)
        northeast_page = northeast_soup['nextPageToken']
        northeast_list = northeast_soup['items']

        full_id_list += get_vid_ids(west_list)
        full_id_list += get_vid_ids(texas_list)
        full_id_list += get_vid_ids(midwest_list)
        full_id_list += get_vid_ids(southeast_list)
        full_id_list += get_vid_ids(northeast_list)
        if track:
            print(i*250 + 250)
    return full_id_list

In [66]:
if __name__ == "__main__":
    num_iterations = 2
    full_vid_list = get_tutorial_url_list(num_iterations,track=True)
    print(full_vid_list)
    


Number of results so far:
250
500
[{'videoID': '86vIYoC5gks', 'title': 'How To live Blessed🙏🏾 🎉 #shorts #youtubershorts'}, {'videoID': 'B_xNI6tuOeI', 'title': 'How to Widen your Driveway'}, {'videoID': 'ianApRDflh0', 'title': 'How to Plant Potatoes! 🥔🌿 // Garden Answer'}, {'videoID': 'K4a3dTScBlg', 'title': 'How To Make A Composting Toilet'}, {'videoID': 'U1fO71zyebs', 'title': '#shorts     How to Be Blessed 🙏🏾'}, {'videoID': 'MTt_Mlj6YN8', 'title': 'HOW TO MAKE OAT MILK | not slimy + secret trick'}, {'videoID': '55z5RSJpzZY', 'title': '#shorts  How to Keep your Blessings'}, {'videoID': 'pz2IupVGVbU', 'title': 'How to Install Lath for Stucco'}, {'videoID': 'NjksWMuLGvY', 'title': 'How to Make Floating Picture Frames'}, {'videoID': '-mUeNy0rweM', 'title': 'How to Grow Ginger in Containers And Get a Huge Harvest'}, {'videoID': 'X_oNQX4srHc', 'title': 'How to Perform Step Ups (Glute Focused)'}, {'videoID': 'G1AoEBvt4_o', 'title': 'PICO DE GALLO | How to Make Authentic Mexican Salsa Recipe

In [74]:
#delete this and relevant packages later, just for testing
df = pd.DataFrame(full_vid_list)
df.to_json('500_videos.json')